In [94]:
import os
import json
import re
from datetime import datetime
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/nazimaniltepe/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [101]:
json_data = []
for path, dirs, files in os.walk(f"./output"):
    for file in files:
        if file.endswith(".json"):
            f = open(os.path.join(path, file), encoding="utf-8-sig")
            json_data.append(json.load(f))
            f.close()
print("number of documents:", len(json_data))

number of documents: 215


In [102]:
text_data = [f"{' '.join([i['text'] for topic in j['topics'] for i in topic['items']])} {j['abstract']}" for j in json_data]
len(text_data)

215

In [103]:
sw_file = open('./stopwords.txt')
stop_words = sw_file.read()
stop_words = stop_words.split("\n")
sw_file.close()
print("stop words count:", len(stop_words))

stop words count: 74


In [104]:
regexes = [
    (r"([’’])", "'"),
    (r"([“”\"])", ""),
    (r"â", "a"),
    (r"î", "i"),
    (r"û", "u"),
    (r"Â", "A"),
    (r"Î", "İ"),
    (r"Û", "U"),
]

In [105]:
text_clean = text_data
for reg in regexes:
    text_clean = [re.sub(reg[0], reg[1], t) for t in text_clean]

text_lower = [t.replace("İ", "i").replace("I", "ı").lower() for t in text_clean]

text_lower = [" ".join([word for word in sent.split(" ") if word not in stop_words]) for sent in text_lower]

In [106]:
regexes2 = [
    (r"(,\s)", " "),
    (r"([\.;\(\)])", ""), 
    (r"['\-]", " "),
]

In [107]:
for reg in regexes2:
    text_lower = [re.sub(reg[0], reg[1], t) for t in text_lower]

text_lower = [" ".join([word for word in sent.split(" ") if word not in stop_words and word != ""]) for sent in text_lower]

In [144]:
unique_words = {}
text_test = text_lower
for w in " ".join(text_test).split(" "):
    unique_words[w] = unique_words[w] + 1 if w in unique_words else 1

In [145]:
len(unique_words)

11820

In [154]:
unique_words_sorted = [(k, unique_words[k]) for k in unique_words]
unique_words_sorted.sort(reverse=True, key=lambda w: w[1])
unique_words_sorted = [w for w in unique_words_sorted if len(w[0]) < 5]

In [112]:
print(len(text_lower))
print(text_lower[0])

215
mayıs ayında tüketici fiyatları yüzde 0,15 oranında artarken yıllık enflasyon yüzde 6,51 yükselmiştir artışta enerji fiyatlarından kaynaklanan baz etkisi belirleyici olurken temel enflasyon göstergeleri ılımlı seyrini korumuştur gıda alkolsüz içecekler grubunda yıllık enflasyon yüzde 7,07 yükselmiştir işlenmiş gıda grubunda fiyatlar mayıs ayında yüzde 0,55 oranında artarken yıllık enflasyon yüzde 8,19 gerilemiştir grup enflasyonunda aylık bazda gözlenen artışın temel belirleyicisi ekmek tahıllar alt grubu olurken diğer işlenmiş gıda fiyatlarının katkısı sınırlı kalmıştır işlenmemiş gıda fiyatları mayıs ayında temelde taze sebze kaynaklı olarak yüzde 5,33 oranında gerilemiş baz etkisi nedeniyle grup yıllık enflasyonu artış kaydetmiştir haziran ayına ilişkin öncü göstergeler grupta yıllık enflasyonunun belirgin olarak yükseleceğine işaret etmektedir uluslararası petrol fiyatlarındaki düşüşün enerji enflasyonuna olumlu yansıması sürmüş grup fiyatları mayıs ayında yüzde 0,69 oranında a

In [155]:
data = [word_tokenize(t, "turkish") for t in sent_tokenize(" ".join(text_lower), "turkish")] 

In [156]:
len(data[0])

374136

In [157]:
model = Word2Vec(data, vector_size=500, window=20, epochs=100)

In [153]:
timestamp = datetime.now().strftime("%y%m%d%H%M")
model.save(f"word2vec_{timestamp}.model")
model.wv.save(f"word2vec_{timestamp}.vectors")

In [147]:
len(model.wv.key_to_index)

4725

In [158]:
model.wv.similar_by_word("işaret")

[('etmektedir', 0.9437370896339417),
 ('ilk', 0.8237730264663696),
 ('süreceğine', 0.7789953947067261),
 ('edeceğine', 0.7697752118110657),
 ('sürdüğüne', 0.7660500407218933),
 ('anket', 0.7598369121551514),
 ('endeksleri', 0.7588145732879639),
 ('ikinci', 0.7570627927780151),
 ('çeyreğine', 0.7512868046760559),
 ('verileri', 0.7422521710395813)]

In [149]:
model.wv.most_similar(positive=["banka", "üretim"], negative=["gerileme"])

[('görülmüştür', 0.37992048263549805),
 ('gösterse', 0.3269602060317993),
 ('yavaşlamakla', 0.3120673596858978),
 ('büyümesi', 0.2894984483718872),
 ('ederken', 0.28884491324424744),
 ('desteklemiştir', 0.2886334955692291),
 ('gözlense', 0.2858155369758606),
 ('endekslerinde', 0.2820065915584564),
 ('firmaların', 0.2740427255630493),
 ('gelişme', 0.27259358763694763)]

In [152]:
model.wv.most_similar(positive=["2022", "mart", "enflasyon"])

[('gıda', 0.6684607267379761),
 ('işlenmemiş', 0.6664918065071106),
 ('sürmüştür', 0.660183846950531),
 ('temel', 0.6598487496376038),
 ('yıllık', 0.6546868681907654),
 ('kalan', 0.6543123126029968),
 ('yükseliş', 0.6447461247444153),
 ('etkili', 0.6388684511184692),
 ('fiyatlarındaki', 0.6241285800933838),
 ('ardından', 0.6206164360046387)]